In [1]:
!nvidia-smi

Mon Dec  9 16:34:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.13                 Driver Version: 556.13         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8              4W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

2.4.1
True


In [3]:
!pip install codecarbon

  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached fief_client-0.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pynvml-12.0.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached questionary-2.0.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jwcrypto-1.5.6-py3-none-any.whl.metadata (3.1 kB)
  Using cached yaspin-3.1.0-py3-none-any.whl.metadata (14 kB)
  Using cached nvidia_ml_py-12.560.30-py3-none-any.whl.metadata (8.6 kB)
  Using cached prompt_toolkit-3.0.36-py3-none-any.whl.metadata (7.0 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.27.0 requires prompt-toolkit<3.1.0,>=3.0.41, but you have prompt-toolkit 3.0.36 which is incompatible.


In [4]:
!mkdir code_carbon

Ya existe el subdirectorio o el archivo code_carbon.


In [3]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker(
    output_dir="./code_carbon/",  # define the directory to which we'll write our emissions results
    output_file="emissions.csv",  # define the name of the file containing our emissions results
    # log_level='error' # comment out this line to see regular output
)
tracker.start()

[codecarbon INFO @ 16:34:35] Codecarbon is taking the configuration from global file: C:\Users\pablo\.codecarbon.config
[codecarbon ERROR @ 16:34:35] Error: Another instance of codecarbon is probably running as we find `C:\Users\pablo\AppData\Local\Temp\.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.
[codecarbon WARNING @ 16:34:35] Another instance of codecarbon is already running. Exiting.


In [1]:
!git clone https://github.com/hidasib/GRU4Rec_PyTorch_Official.git

Cloning into 'GRU4Rec_PyTorch_Official'...


In [2]:
!curl -L -o yoochoose.zip https://www.kaggle.com/api/v1/datasets/download/chadgostopp/recsys-challenge-2015

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0  840M    0 86018    0     0  79868      0  3:03:58  0:00:01  3:03:57 79868
  1  840M    1 14.8M    0     0  7343k      0  0:01:57  0:00:02  0:01:55 14.8M
  4  840M    4 41.4M    0     0  13.4M      0  0:01:02  0:00:03  0:00:59 20.6M
  6  840M    6 57.6M    0     0  14.0M      0  0:00:59  0:00:04  0:00:55 19.0M
  9  840M    9 81.5M    0     0  15.8M      0  0:00:53  0:00:05  0:00:48 20.0M
 12  840M   12  105M    0     0  17.2M      0  0:00:48  0:00:06  0:00:42 20.8M
 15  840M   15  127M    0     0  18.1M      0  0:00:46  0:00:07  0:00:39 22.6M
 17  840M   17  145M    0     0  17.9M      0  0:0

In [3]:
!unzip yoochoose.zip

"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [6]:
!mkdir processed_data

In [4]:
import numpy as np
import pandas as pd
import datetime as dt

PATH_TO_ORIGINAL_DATA = './yoochoose-data/'
PATH_TO_PROCESSED_DATA = './processed_data/'

data = pd.read_csv(PATH_TO_ORIGINAL_DATA + 'yoochoose-clicks.dat', sep=',', header=None, usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})
data.columns = ['SessionId', 'TimeStr', 'ItemId']
data['Time'] = data.TimeStr.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp()) #This is not UTC. It does not really matter.
del(data['TimeStr'])

session_lengths = data.groupby('SessionId').size()
data = data[np.in1d(data.SessionId, session_lengths[session_lengths>1].index)]
item_supports = data.groupby('ItemId').size()
data = data[np.in1d(data.ItemId, item_supports[item_supports>=5].index)]
session_lengths = data.groupby('SessionId').size()
data = data[np.in1d(data.SessionId, session_lengths[session_lengths>=2].index)]

tmax = data.Time.max()
session_max_times = data.groupby('SessionId').Time.max()
session_train = session_max_times[session_max_times < tmax-86400].index
session_test = session_max_times[session_max_times >= tmax-86400].index
train = data[np.in1d(data.SessionId, session_train)]
test = data[np.in1d(data.SessionId, session_test)]
test = test[np.in1d(test.ItemId, train.ItemId)]
tslength = test.groupby('SessionId').size()
test = test[np.in1d(test.SessionId, tslength[tslength>=2].index)]
print('Full train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(train), train.SessionId.nunique(), train.ItemId.nunique()))
train.to_csv(PATH_TO_PROCESSED_DATA + 'rsc15_train_full.txt', sep='\t', index=False)
print('Test set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(test), test.SessionId.nunique(), test.ItemId.nunique()))
test.to_csv(PATH_TO_PROCESSED_DATA + 'rsc15_test.txt', sep='\t', index=False)

tmax = train.Time.max()
session_max_times = train.groupby('SessionId').Time.max()
session_train = session_max_times[session_max_times < tmax-86400].index
session_valid = session_max_times[session_max_times >= tmax-86400].index
train_tr = train[np.in1d(train.SessionId, session_train)]
valid = train[np.in1d(train.SessionId, session_valid)]
valid = valid[np.in1d(valid.ItemId, train_tr.ItemId)]
tslength = valid.groupby('SessionId').size()
valid = valid[np.in1d(valid.SessionId, tslength[tslength>=2].index)]
print('Train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(train_tr), train_tr.SessionId.nunique(), train_tr.ItemId.nunique()))
train_tr.to_csv(PATH_TO_PROCESSED_DATA + 'rsc15_train_tr.txt', sep='\t', index=False)
print('Validation set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(valid), valid.SessionId.nunique(), valid.ItemId.nunique()))
valid.to_csv(PATH_TO_PROCESSED_DATA + 'rsc15_train_valid.txt', sep='\t', index=False)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
!pip install joblib

[codecarbon INFO @ 18:39:46] Energy consumed for RAM : 0.001166 kWh. RAM Power : 5.878415107727051 W
[codecarbon INFO @ 18:39:46] Energy consumed for all GPUs : 0.001765 kWh. Total GPU Power : 6.564417152998804 W
[codecarbon INFO @ 18:39:46] Energy consumed for all CPUs : 0.008532 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:39:46] 0.011463 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:39:46] 0.004593 g.CO2eq/s mean an estimation of 144.8553608786252 kg.CO2eq/year
[codecarbon INFO @ 18:40:01] Energy consumed for RAM : 0.001191 kWh. RAM Power : 5.878415107727051 W
[codecarbon INFO @ 18:40:01] Energy consumed for all GPUs : 0.001795 kWh. Total GPU Power : 7.258350349362 W
[codecarbon INFO @ 18:40:01] Energy consumed for all CPUs : 0.008709 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:40:01] 0.011695 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:40:16] Energy consumed for RAM : 0.001215 kWh. RAM Power : 5.878415107727051 W
[codecarbo

In [5]:
import torch;  num_of_gpus = torch.cuda.device_count(); print(num_of_gpus);

1


In [ ]:
!python GRU4Rec_PyTorch_Official/run.py ./processed_data/rsc15_train_full.txt -t ./processed_data/rsc15_test.txt -m 1 5 10 20 -ps layers=224,batch_size=4126,dropout_p_embed=0.5,dropout_p_hidden=0.05,learning_rate=0.05,momentum=0.4,n_sample=1024,sample_alpha=0.4,bpreg=1.95,logq=0.0,loss=bpr-max,constrained_embedding=True,elu_param=0.5,n_epochs=10 -d cuda:0 -s ./model.pt

In [6]:
tracker._geo

AttributeError: 'EmissionsTracker' object has no attribute '_geo'

In [ ]:
emissions = tracker.stop()
print(f"Emissions from this training run: {emissions:5f} kg CO2eq")

[codecarbon WARNING @ 12:55:13] Another instance of codecarbon is already running. Exiting.


TypeError: unsupported format string passed to NoneType.__format__